In [1]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
lease_num = int('057852')
# 195658

In [3]:
target_url = 'https://webapps2.rrc.texas.gov/EWA/drillingPermitsQueryAction.do'

In [4]:
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Host': 'webapps2.rrc.texas.gov',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'sec-ch-ua': "Google Chrome;v='119', 'Chromium';v='119', 'Not?A_Brand';v='24'",
    'sec-ch-ua-mobile': "?0",
    'sec-ch-ua-platform': "Windows"
}

In [5]:
payload = {
    'methodToCall': 'search',
    'searchArgs.operatorNameWildcardHndlr.inputValue': 'beginsWith',
    'searchArgs.leaseNameWildcardHndlr.inputValue': 'beginsWith',
    'searchArgs.leaseNoHndlr.inputValue': lease_num,
    'searchArgs.fieldNameWildcardHndlr.inputValue': 'beginsWith',
    'searchArgs.surveyNameWildcardHndlr.inputValue': 'beginsWith'
}

In [6]:
try:
    with re.session() as s:
        res_get = s.get(url=target_url,headers=headers,verify=False)
        print('get_code',res_get.status_code)

        if res_get.status_code == 200:
            cookies = res_get.cookies
            res_post = s.post(url=target_url,data=payload,cookies=cookies,headers=headers)

except Exception as e:
    print(e)

c:\Users\Apoorva.Saxena\OneDrive - Sitio Royalties\Desktop\Project - Apoorva\Python\Scraping\RRC\scraper\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webapps2.rrc.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


get_code 200


c:\Users\Apoorva.Saxena\OneDrive - Sitio Royalties\Desktop\Project - Apoorva\Python\Scraping\RRC\scraper\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webapps2.rrc.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [7]:
soup = BeautifulSoup(res_post.text,'html.parser')

In [8]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html>
 <head>
  <meta content="text/html; charset=utf-8;" http-equiv="Content-Type"/>
  <meta content="IE=5, IE=8, IE=9, IE=10" http-equiv="X-UA-Compatible"/>
  <link href="/EWA/styles/ogm_style3.css" rel="stylesheet" type="text/css"/>
  <script type="text/javascript">
   var dojoConfig = {
      parseOnLoad: true,
      isDebug: false,

      has: {
      "dojo-debug-messages": false
      },

      packages:
       [ { name:"rrcGisModules", location:location.pathname.replace(/\/[^/]+$/, '') + '/gis/javascript'} ]

      };
  </script>
  <script src="http://serverapi.arcgisonline.com/jsapi/arcgis/?v=3.3compact" type="text/javascript">
  </script>
  <script type="text/javascript">
   require(["dojo/parser", "dijit/layout/BorderContainer", "dijit/layout/ContentPane", "dojo/query", 
               "dojo/dom-construct", "dojo/NodeList-manipulate", "dojo/dom-attr", "dojo/dom",
   

In [9]:
df = pd.read_html(res_post.text)[9]

C:\Users\Apoorva.Saxena\AppData\Local\Temp\ipykernel_22720\696227653.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(res_post.text)[9]


In [10]:
df.columns = df.iloc[1,:]

In [11]:
df = df.drop([0,1]).copy()

In [14]:
df

1,API NO.,District,Lease,Well Number,Permitted Operator,County,Status Date,Status Number,Wellbore Profiles,Filing Purpose,Amend,Total Depth,Stacked Lateral Parent Well DP #,Status
2,22741470 Links Images GIS Viewer Completion,08,VLT THOR STATE UNIT (57852),0162WB,CHEVRON U. S. A. INC.(148113),HOWARD,Submitted: 12/06/2021 Approved: 12/08/2021,875068,Horizontal,New Drill,N,9000,NaN,APPROVED
3,22741471 Links Images GIS Viewer Completion,08,VLT THOR STATE UNIT (57852),0163WA,CHEVRON U. S. A. INC.(148113),HOWARD,Submitted: 12/06/2021 Approved: 12/08/2021,875070,Horizontal,New Drill,N,9000,875068,APPROVED
4,22741473 Links Images GIS Viewer Completion,08,VLT THOR STATE UNIT (57852),0164WA,CHEVRON U. S. A. INC.(148113),HOWARD,Submitted: 12/06/2021 Approved: 12/08/2021,875073,Horizontal,New Drill,N,9000,875072,APPROVED
5,22741472 Links Images GIS Viewer Completion,08,VLT THOR STATE UNIT (57852),0165WB,CHEVRON U. S. A. INC.(148113),HOWARD,Submitted: 12/06/2021 Approved: 12/08/2021,875072,Horizontal,New Drill,N,9000,NaN,APPROVED
6,22741469 Links Images GIS Viewer Completion,08,VLT THOR STATE UNIT (57852),0161WA,CHEVRON U. S. A. INC.(148113),HOWARD,Submitted: 12/06/2021 Approved: 12/08/2021,875067,Horizontal,New Drill,N,9000,NaN,APPROVED
7,22741493 Links Images GIS Viewer Completion,08,VLT THOR STATE UNIT (57852),0181WA,CHEVRON U. S. A. INC.(148113),HOWARD,Submitted: 12/15/2021 Approved: 12/21/2021,875338,Horizontal,New Drill,N,9000,NaN,APPROVED
8,22741494 Links Images GIS Viewer Completion,08,VLT THOR STATE UNIT (57852),0182WA,CHEVRON U. S. A. INC.(148113),HOWARD,Submitted: 12/15/2021 Approved: 12/21/2021,875339,Horizontal,New Drill,N,9000,NaN,APPROVED
9,22741495 Links Images GIS Viewer Completion,08,VLT THOR STATE UNIT (57852),0183WA,CHEVRON U. S. A. INC.(148113),HOWARD,Submitted: 12/15/2021 Approved: 12/21/2021,875341,Horizontal,New Drill,N,9000,NaN,APPROVED
